# Persisting data - Named Volumes

## Remove unused volumes

In [6]:
docker volume prune -f

## Create the named volume

In [7]:
docker volume create sqlserver

## Verify named volume is there

In [8]:
docker volume ls

## Spin up container with volume mapped

In [9]:
docker container run -d -p 15999:1433 `
--volume sqlserver:/var/opt/sqlserver `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--env MSSQL_DATA_DIR=/var/opt/sqlserver `
--env MSSQL_LOG_DIR=/var/opt/sqlserver `
--name testcontainer9 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Check the container is running

In [10]:
docker container ls -a

## Grant permissions to SQL on the volume

In [13]:
docker exec -u 0 testcontainer9 bash -c "chown mssql /var/opt/sqlserver"

## Create database on volume

In [14]:
mssql-cli -S localhost,15999 -U sa -P Testing1122 `
-Q "CREATE DATABASE [testdatabase];"

## Check that the database is there

In [15]:
mssql-cli -S localhost,15999 -U sa -P Testing1122 `
-Q "SELECT name FROM sys.databases;"

## Blow away the container

In [16]:
docker container kill testcontainer9
docker container rm testcontainer9

## Confirm that the container is gone

In [17]:
docker container ls -a

## Check that the named volume is still there

In [18]:
docker volume ls

## Spin up another container

In [19]:
docker container run -d -p 16100:1433 `
--volume sqlserver:/var/opt/sqlserver `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--env MSSQL_DATA_DIR=/var/opt/sqlserver `
--env MSSQL_LOG_DIR=/var/opt/sqlserver `
--name testcontainer10 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Verify container is running

In [20]:
docker container ls -a

## Check that the database files are there

In [21]:
docker container exec testcontainer10 bash -c "ls -al /var/opt/sqlserver"

## Re-create database

In [22]:
mssql-cli -S localhost,16100 -U sa -P Testing1122 `
-Q "CREATE DATABASE [testdatabase] ON 
(FILENAME = '/var/opt/sqlserver/testdatabase.mdf'), 
(FILENAME = '/var/opt/sqlserver/testdatabase_log.ldf') FOR ATTACH;"

## Check that the database is there

In [23]:
mssql-cli -S localhost,16100 -U sa -P Testing1122 `
-Q "SELECT name FROM sys.databases;"

## Clean up

In [24]:
docker container kill testcontainer10
docker container rm testcontainer10
docker volume rm sqlserver
docker volume prune -f

## It's all a bit manual though...

In [25]:
docker volume create mssqlsystem
docker volume create mssqluser

## Confirm the volumes are there

In [26]:
docker volume ls

## Spin up a container with the volumes mapped

In [27]:
docker container run -d -p 16110:1433 `
--volume mssqlsystem:/var/opt/mssql `
--volume mssqluser:/var/opt/sqlserver `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--env MSSQL_DATA_DIR=/var/opt/sqlserver `
--env MSSQL_LOG_DIR=/var/opt/sqlserver `
--name testcontainer11 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Check that the container is running

In [28]:
docker container ls -a

## Grant SQL access to volume

In [29]:
docker exec -u 0 testcontainer11 bash -c "chown mssql /var/opt/sqlserver"

## Create a database on the volume

In [30]:
mssql-cli -S localhost,16110 -U sa -P Testing1122 `
-Q "CREATE DATABASE [testdatabase]"

## Check that the database is there

In [34]:
mssql-cli -S localhost,16110 -U sa -P Testing1122 `
-Q "SELECT name FROM sys.databases;"

## Blow away the container

In [35]:
docker kill testcontainer11
docker rm testcontainer11

## Confirm that the container is gone

In [36]:
docker container ls -a

## Confirm that the volumes are still there

In [37]:
docker volume ls

## Spin up another container with the volumes mapped

In [38]:
docker container run -d -p 16120:1433 `
--volume mssqlsystem:/var/opt/mssql `
--volume mssqluser:/var/opt/sqlserver `
--env ACCEPT_EULA=Y `
--env SA_PASSWORD=Testing1122 `
--env MSSQL_DATA_DIR=/var/opt/sqlserver `
--env MSSQL_LOG_DIR=/var/opt/sqlserver `
--name testcontainer12 `
mcr.microsoft.com/mssql/server:2019-GDR1-ubuntu-16.04

## Confirm database is there

In [40]:
mssql-cli -S localhost,16120 -U sa -P Testing1122 `
-Q "SELECT name FROM sys.databases;"

## Clean up

In [41]:
docker container kill testcontainer12
docker container rm testcontainer12
docker volume rm mssqlsystem
docker volume rm mssqluser